In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from transformers import BertTokenizer, AutoModel

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.utils import load_config, seed_everything, get_embed

/opt/anaconda3/envs/glycan/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/envs/glycan/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/

In [3]:
# for reproducibility
seed_everything(42)

# config file
config = load_config("../configs/config.yaml")

# Set Seaborn style
sns.set(style=config['sns_params']['style'],
        palette=config['sns_params']['palette'],
        font_scale=config['sns_params']['font_scale'])

# Set Matplotlib parameters
plt.rcParams.update(config['plt_params'])

### Read files

In [4]:
df_glycan = pd.read_pickle(config['paths']['df_glycan_path'])
df_species = pd.read_pickle(config['paths']['df_species_path'])
df_bind = pd.read_pickle(config['paths']['glycan_binding_path'])
df_glycan_list = pd.read_csv(config['paths']['glycan_list_path'])
df_Nglycan = pd.read_pickle(config['paths']['N_glycans_df_path'])


In [ ]:
all_glycan_seqs = list(set(df_glycan['glycan'].tolist() +
                           df_species['glycan'].tolist() +
                           df_bind.columns.tolist() +
                           df_glycan_list['glycan'].tolist() +
                           df_Nglycan['glycan'].tolist()))

In [7]:
df_bind.columns.tolist()

['3-Anhydro-Gal(a1-3)Gal(b1-4)3-Anhydro-Gal(a1-3)Gal4S',
 '3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S',
 '3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S',
 '3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S',
 '3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal2S(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S',
 '3dGal(b1-3)[Fuc(a1-4)]Glc',
 '3dGal(b1-4)Glc',
 '4d8dNeu5Ac(a2-3)Gal(b1-4)Glc',
 '4dNeu5Ac(a2-3)Gal(b1-4)Glc',
 '7dNeu5Ac(a2-3)Gal(b1-4)Glc',
 '8dNeu5Ac(a2-3)Gal(b1-4)Glc',
 '9dNeu5Ac(a2-3)Gal(b1-4)Glc',
 'Ara(a1-5)Ara(a1-5)Ara(a1-5)Ara(a1-5)Ara(a1-5)Ara',
 'Ara(a1-5)Ara(a1-5)Ara(a1-5)Ara(a1-5)Ara(a1-5)Ara(a1-5)Ara',
 'Fuc(a1-2)Gal',
 'Fuc(a1-2)Gal(b1-3)GalNAc',
 'Fuc(a1-2)Gal(b1-3)GalNAc(a1-3)[Fuc(a1-2)]Gal(b1-4)Glc',
 'Fuc(a1-2)Gal(b1-3)GalNAc(a1-3)[Fuc(a1-2)]Gal(b1-4)GlcNAc',
 'Fuc(a1-2)Gal(b1-3)GalNAc(b1-3)Gal',
 'Fuc(a1-2)Gal(b1-3)GalNAc(b1-3)Gal(a1-4)Gal(b1-4)Glc',
 'Fuc

### Function for embedding calculation

In [ ]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(config['paths']['tokenizer'])

# Load the trained model
model = AutoModel.from_pretrained(config['paths']['model'] + 'checkpoint-48070/')

# calculate embeddings
for idx, row in 